In [1]:
import pandas as pd
import numpy as np
import seaborn as sb

#import monai

In [2]:
names = pd.read_csv("archive/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/name_mapping.csv")

In [3]:
names.head()

,Grade,BraTS_2017_subject_ID,BraTS_2018_subject_ID,TCGA_TCIA_subject_ID,BraTS_2019_subject_ID,BraTS_2020_subject_ID
0,HGG,Brats17_CBICA_AAB_1,Brats18_CBICA_AAB_1,NaN,BraTS19_CBICA_AAB_1,BraTS20_Training_001
1,HGG,Brats17_CBICA_AAG_1,Brats18_CBICA_AAG_1,NaN,BraTS19_CBICA_AAG_1,BraTS20_Training_002
2,HGG,Brats17_CBICA_AAL_1,Brats18_CBICA_AAL_1,NaN,BraTS19_CBICA_AAL_1,BraTS20_Training_003
3,HGG,Brats17_CBICA_AAP_1,Brats18_CBICA_AAP_1,NaN,BraTS19_CBICA_AAP_1,BraTS20_Training_004
4,HGG,Brats17_CBICA_ABB_1,Brats18_CBICA_ABB_1,NaN,BraTS19_CBICA_ABB_1,BraTS20_Training_005


In [4]:
survivals = pd.read_csv("archive/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/survival_info.csv")

In [5]:
survivals.head()

,Brats20ID,Age,Survival_days,Extent_of_Resection
0,BraTS20_Training_001,60.463,289,GTR
1,BraTS20_Training_002,52.263,616,GTR
2,BraTS20_Training_003,54.301,464,GTR
3,BraTS20_Training_004,39.068,788,GTR
4,BraTS20_Training_005,68.493,465,GTR


In [6]:
valid_names = pd.read_csv("archive/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/name_mapping_validation_data.csv")

In [7]:
valid_names.head()

,BraTS_2017_subject_ID,BraTS_2018_subject_ID,TCGA_TCIA_subject_ID,BraTS_2019_subject_ID,BraTS_2020_subject_ID
0,Brats17_CBICA_AAM_1,Brats18_CBICA_AAM_1,NaN,BraTS19_CBICA_AAM_1,BraTS20_Validation_001
1,Brats17_CBICA_ABT_1,Brats18_CBICA_ABT_1,NaN,BraTS19_CBICA_ABT_1,BraTS20_Validation_002
2,Brats17_CBICA_ALA_1,Brats18_CBICA_ALA_1,NaN,BraTS19_CBICA_ALA_1,BraTS20_Validation_003
3,Brats17_CBICA_ALT_1,Brats18_CBICA_ALT_1,NaN,BraTS19_CBICA_ALT_1,BraTS20_Validation_004
4,Brats17_CBICA_ALV_1,Brats18_CBICA_ALV_1,NaN,BraTS19_CBICA_ALV_1,BraTS20_Validation_005


In [8]:
valid_survivals = pd.read_csv("archive/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/survival_evaluation.csv")

In [9]:
valid_survivals.head()

,BraTS20ID,Age,ResectionStatus
0,BraTS20_Validation_001,68.170,GTR
1,BraTS20_Validation_002,50.153,GTR
2,BraTS20_Validation_004,21.726,GTR
3,BraTS20_Validation_006,55.482,GTR
4,BraTS20_Validation_007,64.753,GTR


In [10]:
import os
import csv
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import h5py

In [11]:
def imageGenerator():
    input_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\data'
    output_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\converted_data'
    datas = []
    img_size = (512,512)
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.mat'):
            file_path = os.path.join(input_folder,file_name)
            
            with h5py.File(file_path, 'r') as f:
                image = np.array(f['cjdata/image']).T
                label = int(np.array(f['cjdata/label'])[0,0])
    
            image = image.astype(np.float64)
            im_norm = 255*(image-image.min())/(image.max()-image.min())
            im_uint8 = im_norm.astype(np.uint8)

            #label_folder = os.path.join(output_folder,str(label))
            #os.makedirs(label_folder,exist_ok=True)

            #folder = os.path.join(output_folder,label_folder)
            #file_name_base = os.path.splitext(file_name)[0]
            #output_file_path = os.path.join(output_folder,file_name_base + '.jpg')
    
            im_pil = Image.fromarray(im_uint8)
            im_pil = im_pil.resize(img_size)
            datas.append((np.array(im_pil,dtype=np.float32),label))
            #if output_folder.exist:
                #output_folder.replace
            #im_pil.save(output_file_path)
    return datas

In [12]:
data_list = []
alldata_input_folder = r'C:\Users\marto\Desktop\Szakdolgozat\pogram\brainTumor\data'
for file_name in os.listdir(alldata_input_folder): 
    if file_name.endswith('.mat'):
        file_path = os.path.join(alldata_input_folder,file_name) 
        with h5py.File(file_path, 'r') as f:
            data = f['cjdata']
            
            pid = ''.join(chr(c[0]) for c in np.array(data['PID']))
            image = np.array(data['image']).T
            label = int(np.array(data['label'])[0][0])  #1 for meningioma, 2 for glioma, 3 for pituitary tumor         
            tumor_border = np.array(data['tumorBorder']).T
            tumor_mask = np.array(data['tumorMask']).T

            data_structure = {
                'PID': pid,
                'image': image,
                'label': label,
                'tumorBorder': tumor_border,
                'tumormask': tumor_mask
            }

            data_list.append(data_structure)
    

In [13]:
data_list[0]

{'PID': '100360',
 'image': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'label': 1,
 'tumorBorder': array([[267.61524501],
        [231.37568058],
        [277.83666062],
        [248.10163339],
        [289.91651543],
        [250.8892922 ],
        [305.71324864],
        [253.676951  ],
        [318.72232305],
        [249.9600726 ],
        [321.50998185],
        [237.88021779],
        [354.96188748],
        [234.16333938],
        [367.97096189],
        [227.65880218],
        [380.9800363 ],
        [210.93284936],
        [372.61705989],
        [195.13611615],
        [363.32486388],
        [177.48094374],
        [350.31578947],
        [176.55172414],
        [336.37749546],
        [183.98548094],
        [317.79310345],
        [196.06533575],
        [305.71324864],
        [207.21597096],

In [14]:
data_frame = pd.DataFrame(data_list)

In [15]:
data_frame.head()

,PID,image,label,tumorBorder,tumormask
0,100360,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,"[[267.6152450090744], [231.37568058076226], [2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,101016,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,"[[248.86411149825784], [256.89198606271776], [...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,107494,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,"[[193.26370732550265], [175.8076305348121], [1...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,112649,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3,"[[225.95188511210213], [231.1718956592464], [2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,112649,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3,"[[231.61100841767436], [237.4012836281162], [2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [ ]:
da